In [1]:
# ...existing code...
import tensorflow as tf
import numpy as np
from tensorflow.keras import layers, models
from tensorflow.keras.applications import mobilenet_v2
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau

AUTOTUNE = tf.data.AUTOTUNE
IMG_SIZE = 160
BATCH = 32
EPOCHS_HEAD = 10
EPOCHS_FT = 15
NUM_CLASSES = 10
# ...existing code...

In [2]:
# ...existing code...
gpus = tf.config.list_physical_devices('GPU')
for g in gpus:
    try:
        tf.config.experimental.set_memory_growth(g, True)
    except Exception:
        pass
# ...existing code...

In [3]:
# ...existing code...
# 1) Load CIFAR-10
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()
y_train = y_train.squeeze().astype("int32")
y_test = y_test.squeeze().astype("int32")

# Split validasi
VAL_SPLIT = 5000
x_val, y_val = x_train[-VAL_SPLIT:], y_train[-VAL_SPLIT:]
x_train, y_train = x_train[:-VAL_SPLIT], y_train[:-VAL_SPLIT]
# ...existing code...

In [4]:
# ...existing code...
# 2) Pipeline tf.data: resize + augment + preprocess
def resize(img, label):
    img = tf.image.resize(img, (IMG_SIZE, IMG_SIZE))
    return img, label

augment = tf.keras.Sequential([
    layers.RandomFlip("horizontal"),
    layers.RandomRotation(0.05),
    layers.RandomZoom(0.1),
])

def aug_and_preprocess(img, label):
    img = augment(img)
    img = mobilenet_v2.preprocess_input(img)  # [-1,1] sesuai MobileNetV2
    return img, label

def preprocess_only(img, label):
    img = mobilenet_v2.preprocess_input(img)
    return img, label

# ...existing code...
def make_ds(x, y, training=True):
    ds = tf.data.Dataset.from_tensor_slices((x, y))
    ds = ds.shuffle(10000) if training else ds
    ds = ds.map(resize, num_parallel_calls=AUTOTUNE)
    ds = ds.map(aug_and_preprocess if training else preprocess_only, num_parallel_calls=AUTOTUNE)
    # return ds.batch(BATCH).cache().prefetch(AUTOTUNE)  # memakai cache di memori -> bisa OOM
    return ds.batch(BATCH).prefetch(AUTOTUNE)
# ...existing code...
# ...existing code...

2025-11-09 22:35:54.309290: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M2
2025-11-09 22:35:54.309316: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 16.00 GB
2025-11-09 22:35:54.309333: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 5.92 GB
2025-11-09 22:35:54.309418: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:303] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2025-11-09 22:35:54.309468: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:269] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [5]:
# ...existing code...
train_ds = make_ds(x_train, y_train, training=True)
val_ds   = make_ds(x_val, y_val, training=False)
test_ds  = make_ds(x_test, y_test, training=False)
# ...existing code...

In [6]:
# ...existing code...
# 3) Bangun model: MobileNetV2 backbone + head kecil
base = mobilenet_v2.MobileNetV2(
    input_shape=(IMG_SIZE, IMG_SIZE, 3),
    include_top=False,
    weights="imagenet"
)
base.trainable = False  # tahap 1: freeze backbone

inputs = layers.Input(shape=(IMG_SIZE, IMG_SIZE, 3))
x = inputs
# (augment sudah di pipeline tf.data)
x = base(x, training=False)
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dropout(0.2)(x)
# Bila mixed precision aktif, pastikan output dtype float32
outputs = layers.Dense(NUM_CLASSES, activation="softmax", dtype="float32")(x)
model = models.Model(inputs, outputs)

model.compile(
    optimizer=tf.keras.optimizers.Adam(1e-3),
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"]
)

model.summary()
# ...existing code...

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 160, 160, 3)]     0         
                                                                 
 mobilenetv2_1.00_160 (Func  (None, 5, 5, 1280)        2257984   
 tional)                                                         
                                                                 
 global_average_pooling2d (  (None, 1280)              0         
 GlobalAveragePooling2D)                                         
                                                                 
 dropout (Dropout)           (None, 1280)              0         
                                                                 
 dense (Dense)               (None, 10)                12810     
                                                                 
Total params: 2270794 (8.66 MB)
Trainable params: 12810 (50.0

In [7]:
# ...existing code...
# Callbacks
ckpt = ModelCheckpoint("mobilenetv2_cifar10_head.keras", monitor="val_accuracy",
                       save_best_only=True, mode="max", verbose=1)
early = EarlyStopping(patience=5, restore_best_weights=True, monitor="val_accuracy")
plateau = ReduceLROnPlateau(monitor="val_loss", factor=0.5, patience=2, verbose=1)

# ...existing code...

In [8]:

# 4) Tahap 1: train classification head
history_head = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=EPOCHS_HEAD,
    callbacks=[ckpt, early, plateau],
    verbose=1,
)

Epoch 1/10


2025-11-09 22:36:07.762339: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2025-11-09 22:36:07.800097: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2025-11-09 22:36:08.620718: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


   7/1407 [..............................] - ETA: 1:49 - loss: 2.6427 - accuracy: 0.1875

: 

In [ ]:
# ...existing code...
# 5) Tahap 2: Fine-tune sebagian backbone
base.trainable = True
# Unfreeze dari blok tertentu agar stabil (angka 100 cocok untuk MobileNetV2)
fine_tune_at = 100
for i, layer in enumerate(base.layers):
    layer.trainable = (i >= fine_tune_at)

model.compile(
    optimizer=tf.keras.optimizers.Adam(1e-4),
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"],
)

ckpt_ft = ModelCheckpoint("mobilenetv2_cifar10_finetune.keras", monitor="val_accuracy",
                          save_best_only=True, mode="max", verbose=1)
early_ft = EarlyStopping(patience=6, restore_best_weights=True, monitor="val_accuracy")

history_ft = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=EPOCHS_FT,
    callbacks=[ckpt_ft, early_ft, plateau],
    verbose=1,
)

# 6) Evaluasi di test set
test_loss, test_acc = model.evaluate(test_ds, verbose=0)
print(f"Test accuracy: {test_acc:.4f}")
# ...existing code...